In [1]:
import json
import sys
from typing import Any, Dict, List

sys.path.append('/Users/yujunwon/Project/dancify/ai/yujun/complete')           # 경로 설정 필요

import angle_calc

## 1. 데이터 로드

In [2]:
def load_keypoints_from_json(file_path: str) -> List[Dict[str, Any]]:
    """
    주어진 파일 경로로부터 JSON 파일을 읽고, 키포인트 정보를 추출하여 리스트로 반환합니다.

    Args:
        file_path (str): JSON 파일의 경로

    Returns:
        List[Dict[str, Any]]: 키포인트 정보를 담고 있는 딕셔너리의 리스트
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
        keypoints = angle_calc.get_keypoints(data)
    return keypoints

### 고려할 점
1. dancer_joint와 danceable_joint 중에서 하나에 맞춰서 비교해야 함 → 하나에 맞춰서 일단 구현
2. 프레임 단위로 비교할 때, 앞뒤 5프레임 정도를 평균내서 하는 것은 어떨지에 대해서 생각해보기 → 내일(0615) 시도해보기
3. 리스트의 맨 끝에 프레임 번호를 추가할깝쇼... → 내일(0615) 물어보기

In [3]:
def calculate_angle_difference(dancer_json_path, danceable_json_path):
    dancer = load_keypoints_from_json(dancer_json_path)
    danceable = load_keypoints_from_json(danceable_json_path)

    dancer_joint = angle_calc.calculate_joint_angles(dancer)
    danceable_joint = angle_calc.calculate_joint_angles(danceable)

    joint_list = ['left_pelvic_joint', 'right_pelvic_joint', 'left_shoulder_joint', 'right_shoulder_joint',
                  'left_elbow_joint', 'right_elbow_joint', 'left_knee_joint', 'right_knee_joint']

    diff_list = []

    for dancer_joint_item, danceable_joint_item in zip(dancer_joint, danceable_joint):
        diff = [abs(dancer_joint_item[joint] - danceable_joint_item[joint]) for joint in joint_list]
        diff_list.append(list(diff))

    return diff_list

In [7]:
result = calculate_angle_difference("../point_sample/chansol.json", "../point_sample/heewon.json")
result

[[34.691423932402174,
  12.573530875042366,
  47.88162516359236,
  13.578248288536834,
  21.697466533750358,
  14.275977551941743,
  45.77824556516555,
  4.724209764125732]]